In [2]:
# File path
examples_file_path = 'ASK-groundtruth-v3.jsonl'

#### Read the Eval set and parse it according to your ASK Eval set schema

In [1]:
import pandas as pd
import json

# Read the JSONL file and convert it into a DataFrame
data = []
with open(examples_file_path, 'r') as f:
    for line in f:
        data.append(json.loads(line))

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(
    {
        "metadata": [item["metadata"]["dataset_split"] for item in data],
        "inputs_question": [item["inputs"]["question"] for item in data],
        "outputs_ground_truth_answer": [
            item["outputs"]["ground_truth_answer"] for item in data
        ],
        "outputs_ground_truth_sources": [
            item["outputs"]["ground_truth_sources"] for item in data
        ],
    }
)

NameError: name 'examples_file_path' is not defined

#### **OPTIONAL**: Display the Eval Set in an Editor

In [3]:
import logging
import warnings
import os

os.environ["APPDATA"] = "/tmp"

from pandasgui import show

# Suppress pandasgui logging
logging.getLogger("pandasgui").setLevel(logging.ERROR)
warnings.simplefilter(action='ignore', category=FutureWarning)


gui = show(df)

This process is not trusted! Input event monitoring will not be possible until it is added to accessibility clients.
qt.qpa.fonts: Populating font family aliases took 139 ms. Replace uses of missing font family "Consolas" with one that exists to avoid this cost. 


### Select the LangSmith account to use based on API key

In [3]:
import os, sys
from dotenv import load_dotenv
from langsmith import Client

load_dotenv('/Users/drew_wilkins/Drews_Files/Drew/Python/Localcode/.env')

# choose the Langsmith account you want to use based on the API key
client = Client(api_key=os.environ["LANGCHAIN_API_KEY"])

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


### Upload Eval Set to LangSmith

In [5]:
dataset_name = "ASK-groundtruth-v3"

2025-01-28 17:00:23.260 Python[23191:249021] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-28 17:00:23.260 Python[23191:249021] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [6]:
dataset = client.create_dataset(dataset_name)

# Read and upload data from the JSONL file
with open(examples_file_path, "r") as f:
    for line in f:
        entry = json.loads(line)

        client.create_example(
            dataset_name=dataset_name,
            inputs={"question": entry["inputs"]["question"]},  # Input question
            outputs={
                # Ground truth answer
                "ground_truth_answer": entry["outputs"]["ground_truth_answer"],
                # Ground truth sources
                "ground_truth_sources": entry["outputs"]["ground_truth_sources"]
            },
            metadata={
                # Dataset split information
                "dataset_split": entry["metadata"]["dataset_split"]
            }
        )


print(f"Dataset '{dataset_name}' has been successfully uploaded to LangSmith.")

Dataset 'ASK-groundtruth-v3' has been successfully uploaded to LangSmith.


In [7]:
datasets = client.list_datasets()
print("Available Datasets:")
for dataset in datasets:
    print(f"- Name: {dataset.name}, ID: {dataset.id}")

Available Datasets:
- Name: one_example_easy, ID: 4a2a2222-1352-4b8c-8257-a8461e439097
- Name: ASK-groundtruth-v3, ID: e44c3abc-7871-49a3-a388-6197d7c2dcf3


Find out how many traces done

In [14]:
import requests
import os

# Set your LangSmith API key
api_key = os.environ["LANGCHAIN_API_KEY"]

# Define the API endpoints
sessions_url = "https://api.smith.langchain.com/sessions"
runs_url = "https://api.smith.langchain.com/runs/query"

# Set the headers with your API key
headers = {
    "x-api-key": api_key,
    "Content-Type": "application/json"
}

# Function to retrieve all sessions for a project


def get_sessions(project_name):
    params = {"project_name": project_name}
    response = requests.get(sessions_url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(
            f"Failed to retrieve sessions: {response.status_code} - {response.text}")
        return []

# Function to retrieve root runs for a list of session IDs


def get_root_runs(project_name, session_ids):
    payload = {
        "project_name": project_name,
        "is_root": True,
        "session": session_ids  # Pass the list of session IDs
    }
    response = requests.post(runs_url, json=payload, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(
            f"Failed to retrieve runs: {response.status_code} - {response.text}")
        return []


# Main logic
project_name = "evaluators"
sessions = get_sessions(project_name)

if not sessions:
    print("No sessions found for the project.")
    exit()

# Extract session IDs
session_ids = [session["id"] for session in sessions]

# Retrieve root runs for all sessions
root_runs = get_root_runs(project_name, session_ids)

# Output the total number of root runs
if root_runs:
    print(f"Total number of traces in '{project_name}': {len(root_runs)}")
else:
    print("No root runs found.")

Total number of traces in 'evaluators': 3


In [5]:
print(api_key)

None


Get test results

UsageError: Line magic function `%ollama` not found.
